## The Example
Let's keep the example simple since our main foucs is on understanding the APIs. 

The data we'll be using is part of a dataset taken [here](https://www.kaggle.com/datasets/dgawlik/nyse) from Kaggle. This is a time-series data (ordered by time) of Yahoo's highest stock prices per day **(?)**. Our objective is as follows:
"Given the price of the previous 5 days, predict the price on the next (6th) day"

The data has already been cleaned and made ready to be fed into the model.
To be more specific, the $ i^{th} $ training sample, denoted as $ (X^{i}, y^{i}) $, is such that $ X^{i} $ consists of some 5 consecutive days' of prices and $ y^{i} $ denotes the price on the 6th day.


In [1]:
import data.data_extraction as data
import pandas as pd
import numpy as np

X, y = data.get_data(".\data\prices-split-adjusted.csv")
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (1757, 5, 1)
Shape of y: (1757, 1)


`X`, which is going to serve as the input to our final *model*, is of shape `[batch, timestamp, feature]`:
-   `batch` refers to the total number of samples to our model
-   `timestamp` refers to the length of each time-series sample (5 in our case)
-   `feature` refers to the number of features in each timestamp of the sample (which is just 1 in this case - the highest stock value)

Here's a closer look at the training sample:

In [2]:
data.display_samples(X[0:3], y[0:3])

n = 3, T = 5
+--------+--------+-------+---------+
| Sample | Inputs         | Outputs |
+        +--------+-------+---------+
|        | Day    | Price | Price   |
+--------+--------+-------+---------+
| 1      | Day_1  | 17.2  | 16.83   |
+        +--------+-------+         +
|        | Day_2  | 17.23 |         |
+        +--------+-------+         +
|        | Day_3  | 17.3  |         |
+        +--------+-------+         +
|        | Day_4  | 16.9  |         |
+        +--------+-------+         +
|        | Day_5  | 16.76 |         |
+--------+--------+-------+---------+
| 2      | Day_1  | 17.23 | 16.86   |
+        +--------+-------+         +
|        | Day_2  | 17.3  |         |
+        +--------+-------+         +
|        | Day_3  | 16.9  |         |
+        +--------+-------+         +
|        | Day_4  | 16.76 |         |
+        +--------+-------+         +
|        | Day_5  | 16.83 |         |
+--------+--------+-------+---------+
| 3      | Day_1  | 17.3  | 16.98   |

## Let's Train!
We'll use the `Sequential API` to build and train our RNN model

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [4]:
y_train.shape

(1229, 1)

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

In [6]:
# parameters
n_inputs = 1  # number of features
n_neurons = 25
n_outputs = 1
n_layers = 2
batch_size = 32
n_epochs = 25

In [14]:
RNNmodel = Sequential()
RNNmodel.add(
    layers.LSTM(n_neurons)
)
# RNNmodel.add(
#     layers.RNN(
#         layers.StackedRNNCells(
#             [
#                 layers.LSTMCell(n_neurons) for _ in range(n_layers)
#             ]
#         )
#     )
# )
RNNmodel.add(layers.Dense(n_outputs, activation='linear'))

RNNmodel.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(),
)

RNNmodel.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test), 
    batch_size=batch_size, 
    epochs=n_epochs,
)

Epoch 1/25
39/39 [==============================] - 3s 21ms/step - loss: 888.1685 - val_loss: 816.7823
Epoch 2/25
39/39 [==============================] - 0s 7ms/step - loss: 797.4676 - val_loss: 751.3549
Epoch 3/25
39/39 [==============================] - 0s 7ms/step - loss: 707.8855 - val_loss: 624.2955
Epoch 4/25
39/39 [==============================] - 0s 6ms/step - loss: 604.1218 - val_loss: 555.8630
Epoch 5/25
39/39 [==============================] - 0s 6ms/step - loss: 518.2236 - val_loss: 454.6849
Epoch 6/25
39/39 [==============================] - 0s 6ms/step - loss: 433.9761 - val_loss: 394.6521
Epoch 7/25
39/39 [==============================] - 0s 6ms/step - loss: 386.9225 - val_loss: 357.6775
Epoch 8/25
39/39 [==============================] - 0s 6ms/step - loss: 352.9790 - val_loss: 326.6665
Epoch 9/25
39/39 [==============================] - 0s 6ms/step - loss: 324.2498 - val_loss: 300.4897
Epoch 10/25
39/39 [==============================] - 0s 6ms/step - loss: 299.5553

In [15]:
ans = RNNmodel.predict(X_test)

17/17 [==============================] - 0s 2ms/step


In [21]:
# compare predicted and actual value
idx = 14  # change this to see for a different example
print("Predicted: {}, Actual: {}".format(ans[idx], y_test[idx]))

Predicted: [25.036465], Actual: [25.54]


## LSTM

In [ ]:
lstm = tf.keras.layers.LSTM(4, activation='linear', input_shape=(None, 8), return_sequences=True)
output_lstm = lstm(inputs,)

In [ ]:
weights_ = lstm.get_weights()

In [ ]:
from keras.engine import keras_tensor
isinstance(inputs, keras_tensor.KerasTensor)

False

In [ ]:
tf.executing_eagerly()

True

In [ ]:
from keras.layers.rnn.rnn_utils import standardize_args

standardize_args(inputs, None, None, None)

In [ ]:
print(output_lstm[0])

tf.Tensor(
[[ 0.15028077  0.02378442 -0.15648831 -0.18393101]
 [ 0.21888971 -0.06897657 -0.5439828  -0.26890522]], shape=(2, 4), dtype=float32)


In [ ]:
i, f, o, c = tf.split(weights_[0], 4, axis=1)
i_, f_, o_, c_ = tf.split(weights_[1], 4, axis=1)

In [ ]:
h_0 = np.zeros(shape=(4,), dtype=np.float32)
i_1 = np.matmul(inputs[0][0], i) + np.matmul(h_0, i_)
f_1 = np.matmul(inputs[0][0], f) + np.matmul(h_0, f_)
o_1 = np.matmul(inputs[0][0], o) + np.matmul(h_0, o_)
c_t = np.matmul(inputs[0][0], c) + np.matmul(h_0, c_)
c_1 = i_1 * c_t
# h_1 = c_1 * o_1

In [ ]:
c_1 * o_1

array([ 1.0291979e-02, -4.7132911e-04,  2.0280483e-03,  6.0263123e-05],
      dtype=float32)

AttributeError: 'LSTM' object has no attribute 'initial_states'